In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
import copy
import argparse
import itertools
from collections import Counter
from collections import deque

import cv2 as cv
import mediapipe as mp

from tensorflow import keras


In [3]:
data = np.zeros((0,33))
cnt = 0
for j in range(4):
    for i in range(7):
        data_path = f'./data/{j}/point_history_{i}.csv'
        tmp = pd.read_csv(data_path)
        tmp = np.array(tmp)
        cnt += tmp.shape[0]
        data = np.vstack((data,tmp))
data = np.array(data)

In [4]:
label = data[::21,0].flatten()
data = data[:,1:].reshape(4344,1,21,32)
label = keras.utils.to_categorical(label, 7)

In [5]:
# 21X32 -> 16X42 변환
hdata = np.zeros((4344,1,16,42))
for i in range(4344):
    for j in range(32):
        k = 0
        if j%2:
            k=1
        hdata[i,0,j//2,k::2] = data[i,0,:,j]
data = hdata

In [6]:
label = label.reshape(4344,1,7).astype(int)

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((data,label)).shuffle(buffer_size=1000)

In [8]:
dataset = train_dataset

In [9]:
def is_test(x, y):
    return x % 4 == 0

def is_train(x, y):
    return not is_test(x, y)

recover = lambda x,y: y

test_dataset = dataset.enumerate() \
                    .filter(is_test) \
                    .map(recover)

train_dataset = dataset.enumerate() \
                    .filter(is_train) \
                    .map(recover)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [32]:
print(train_dataset)

<MapDataset element_spec=(TensorSpec(shape=(1, 16, 42), dtype=tf.float64, name=None), TensorSpec(shape=(1, 7), dtype=tf.int32, name=None))>


In [45]:
model= tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units = 50, return_sequences=True ,input_shape = (16,42)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7, activation='softmax'),   
    tf.keras.layers.Flatten()                         
])


model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [46]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=10)
check_point = tf.keras.callbacks.ModelCheckpoint(filepath='./motionmodel/{epoch}_{accuracy:.4f}.h5', monitor='loss', save_best_only=True, verbose=1)
model.fit(train_dataset, epochs = 32, callbacks=[es, check_point])

Epoch 1/32
   3253/Unknown - 38s 10ms/step - loss: 1.1650 - accuracy: 0.5893
Epoch 1: loss improved from inf to 1.16441, saving model to ./motionmodel\1_0.5893.h5
3258/3258 [==============================] - 38s 10ms/step - loss: 1.1644 - accuracy: 0.5893
Epoch 2/32
3253/3258 [============================>.] - ETA: 0s - loss: 0.7874 - accuracy: 0.7369
Epoch 2: loss improved from 1.16441 to 0.78731, saving model to ./motionmodel\2_0.7370.h5
3258/3258 [==============================] - 33s 10ms/step - loss: 0.7873 - accuracy: 0.7370
Epoch 3/32
3254/3258 [============================>.] - ETA: 0s - loss: 0.5034 - accuracy: 0.8325
Epoch 3: loss improved from 0.78731 to 0.50308, saving model to ./motionmodel\3_0.8327.h5
3258/3258 [==============================] - 38s 12ms/step - loss: 0.5031 - accuracy: 0.8327
Epoch 4/32
3254/3258 [============================>.] - ETA: 0s - loss: 0.4656 - accuracy: 0.8525
Epoch 4: loss improved from 0.50308 to 0.46545, saving model to ./motionmodel\4_0.85

In [47]:
model = tf.keras.models.load_model('./motionmodel/31_0.9761.h5')

In [48]:
model.evaluate(test_dataset)

1086/1086 [==============================] - 4s 3ms/step - loss: 0.1459 - accuracy: 0.9595


[0.14585533738136292, 0.959484338760376]

In [49]:
model.save('./motionmodel_total.h5')